# Data Mining Final

**Editor:**

Anastasios Bazinis - 1115201500099

## Prepreparation

- Import libraries and packages

In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict, Counter
import re
import sys
import spacy

import nltk
from nltk.tokenize import RegexpTokenizer, word_tokenize,WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag


from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


import matplotlib
import matplotlib.pyplot as plt

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')

%matplotlib inline

   - Create train an test Dataframes <br><br>
**In column Comments**
    - Remove links
    - Replace unnecessary ascii characters with space
    - Convert to lowercase

In [2]:
tr = pd.read_csv("data/train.csv")
te = pd.read_csv("data/impermium_verification_set.csv")

tr['Comment'] = tr['Comment'].str.replace('http\S+|www.\S+', '', case=False)
tr['Comment'] = tr['Comment'].str.lower()

tr['Comment'] = tr['Comment'].str.replace(r'\\x..'," ",regex = True)
tr['Comment'] = tr['Comment'].str.replace(r'\\u....'," ",regex = True)
tr['Comment'] = tr['Comment'].str.replace(r'\\n'," ",regex = True)
tr['Comment'] = tr['Comment'].str.replace(r'\\.'," ",regex = True)


te['Comment'] = te['Comment'].str.replace('http\S+|www.\S+', '', case=False)
te['Comment'] = te['Comment'].str.lower()

te['Comment'] = te['Comment'].str.replace(r'\\x..'," ",regex = True)
te['Comment'] = te['Comment'].str.replace(r'\\u....'," ",regex = True)
te['Comment'] = te['Comment'].str.replace(r'\\n'," ",regex = True)
te['Comment'] = te['Comment'].str.replace(r'\\.'," ",regex = True)


- Make a list of every comment in test set

In [3]:
test_lowercase = []
clean_test_words = []
clean_test_words_str = []
tokenizer = WordPunctTokenizer()
for word in te['Comment']:
    test_lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(test_lowercase)):
    
    clean_test_words.append([token for token in test_lowercase[x] if token.isalpha()])
    clean_test_words_str.append(" ".join(clean_test_words[i]))
    i+=1

- Make a list of every comment in train set

In [4]:
lowercase = []
clean_words = []
clean_words_str = []
tokenizer = WordPunctTokenizer()
for word in tr['Comment']:
    lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(lowercase)):
    
    clean_words.append([token for token in lowercase[x] if token.isalpha()])
    clean_words_str.append(" ".join(clean_words[i]))
    i+=1

#### Perform Naive Bayes classification for word vectors

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(clean_words_str)
T = vectorizer.transform(clean_test_words_str)

X_train, Y_train = X.toarray(), tr['Insult'] 
X_test = T.toarray()
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])


clf = GaussianNB()
clf.fit(X_train,Y_train)

Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))

print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.54      0.51      0.52      1158
           1       0.50      0.53      0.51      1077

    accuracy                           0.52      2235
   macro avg       0.52      0.52      0.52      2235
weighted avg       0.52      0.52      0.52      2235

0.5181208053691275
0.5182302071383941


## Classification

### Lemmatization Only

In [6]:
lemmatizer = WordNetLemmatizer()
test_lowercase = []
clean_test_words1 = []
clean_test_words_str1 = []
lemmatized_output_test = []
tokenizer = WordPunctTokenizer()

for word in te['Comment']:
    test_lowercase.append(tokenizer.tokenize(word.lower()))

    
i=0
for x in range(0,len(test_lowercase)):
    
    clean_test_words1.append([token for token in test_lowercase[x] if token.isalpha()])
    lemmatized_output_test.append([lemmatizer.lemmatize(w,pos = "v") for w in clean_test_words1[i]])
    clean_test_words_str1.append(" ".join(lemmatized_output_test[i]))
    i+=1



In [7]:
lemmatizer = WordNetLemmatizer()
lowercase = []
clean_words1 = []
clean_words_str1 = []
lemmatized_output = []
tokenizer = WordPunctTokenizer()

for word in tr['Comment']:
    lowercase.append(tokenizer.tokenize(word.lower()))


i=0
for x in range(0,len(lowercase)):
    
    clean_words1.append([token for token in lowercase[x] if token.isalpha()])
    lemmatized_output.append([lemmatizer.lemmatize(w,pos = "v") for w in clean_words1[i]])
    clean_words_str1.append(" ".join(lemmatized_output[i]))
    i+=1


In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(clean_words_str1)
T = vectorizer.transform(clean_test_words_str1)

X_train, Y_train = X.toarray(), tr['Insult'] 
X_test = T.toarray()
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])


clf = GaussianNB()
clf.fit(X_train,Y_train)

Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))

print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.53      0.46      0.49      1158
           1       0.50      0.57      0.53      1077

    accuracy                           0.51      2235
   macro avg       0.52      0.51      0.51      2235
weighted avg       0.52      0.51      0.51      2235

0.512751677852349
0.5112987686488892


### Stopwords Only

In [9]:
stop_words = set(stopwords.words('english'))
    
test_lowercase = []
clean_test_words = []
clean_test_words_str = []
tokenizer = WordPunctTokenizer()
for word in te['Comment']:
    test_lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(test_lowercase)):
    
    clean_test_words.append([token for token in test_lowercase[x] if not token in stop_words and token.isalpha()])
    clean_test_words_str.append(" ".join(clean_test_words[i]))
    i+=1


In [10]:
stop_words = set(stopwords.words('english'))
    
lowercase = []
clean_words = []
clean_words_str = []
tokenizer = WordPunctTokenizer()
for word in tr['Comment']:
    lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(lowercase)):
    
    clean_words.append([token for token in lowercase[x] if not token in stop_words and token.isalpha()])
    clean_words_str.append(" ".join(clean_words[i]))
    i+=1

In [11]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(clean_words_str)
T = vectorizer.transform(clean_test_words_str)

X_train, Y_train = X.toarray(), tr['Insult'] 
X_test = T.toarray()
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])


clf = GaussianNB()
clf.fit(X_train,Y_train)

Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))

print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.54      0.50      0.52      1158
           1       0.50      0.53      0.51      1077

    accuracy                           0.52      2235
   macro avg       0.52      0.52      0.52      2235
weighted avg       0.52      0.52      0.52      2235

0.516331096196868
0.516411270642958


### Bigrams Only

In [12]:
lowercase = []
clean_words = []
clean_words_str = []
tokenizer = WordPunctTokenizer()
for word in tr['Comment']:
    lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(lowercase)):
    
    clean_words.append([token for token in lowercase[x] if token.isalpha()])
    clean_words_str.append(" ".join(clean_words[i]))
    i+=1


In [13]:
test_lowercase = []
clean_test_words = []
clean_test_words_str = []
tokenizer = WordPunctTokenizer()
for word in te['Comment']:
    test_lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(test_lowercase)):
    
    clean_test_words.append([token for token in test_lowercase[x] if token.isalpha()])
    clean_test_words_str.append(" ".join(clean_test_words[i]))
    i+=1

In [14]:
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(clean_words_str)
T = vectorizer.transform(clean_test_words_str)

X_train, Y_train = X.toarray(), tr['Insult'] 
X_test = T.toarray()
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])


clf = GaussianNB()
clf.fit(X_train,Y_train)

Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))

print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.57      0.61      0.59      1158
           1       0.55      0.51      0.53      1077

    accuracy                           0.56      2235
   macro avg       0.56      0.56      0.56      2235
weighted avg       0.56      0.56      0.56      2235

0.5610738255033557
0.5601986774993744


### Laplace Smoothing Only

In [15]:
lowercase = []
clean_words = []
clean_words_str = []
tokenizer = WordPunctTokenizer()
for word in tr['Comment']:
    lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(lowercase)):
    
    clean_words.append([token for token in lowercase[x] if token.isalpha()])
    clean_words_str.append(" ".join(clean_words[i]))
    i+=1

In [16]:
test_lowercase = []
clean_test_words = []
clean_test_words_str = []
tokenizer = WordPunctTokenizer()
for word in te['Comment']:
    test_lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(test_lowercase)):
    
    clean_test_words.append([token for token in test_lowercase[x] if token.isalpha()])
    clean_test_words_str.append(" ".join(clean_test_words[i]))
    i+=1

In [17]:
vectorizer = CountVectorizer()
X2 = vectorizer.fit_transform(clean_words_str)
T2 = vectorizer.transform(clean_test_words_str)

X_train, Y_train = X2.toarray(), tr['Insult'] 
X_test = T2.toarray()
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])

clf = MultinomialNB(alpha=1.0) # Laplace Smoothing
clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))


print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.66      0.78      0.72      1158
           1       0.71      0.58      0.64      1077

    accuracy                           0.68      2235
   macro avg       0.69      0.68      0.68      2235
weighted avg       0.69      0.68      0.68      2235

0.6832214765100671
0.6794093192097572


##### The reason why we get better results in the Multinomial NB than the Gaussian Nb is because the Multinomial NB is used when we have discrete data. In text learning we have the count of each word to predict the class or label.

### Combination of all 4 of the previous characteristics

#### Lemmatization and stopwords elimination of test set

In [18]:

lemmatizer = WordNetLemmatizer()
test_lowercase = []
clean_test_words1 = []
clean_test_words_str1 = []
lemmatized_output_test = []
tokenizer = WordPunctTokenizer()

for word in te['Comment']:
    test_lowercase.append(tokenizer.tokenize(word.lower()))

stop_words = set(stopwords.words('english'))
    
i=0
for x in range(0,len(test_lowercase)):
    
    clean_test_words1.append([token for token in test_lowercase[x] if not token in stop_words and token.isalpha()])
    lemmatized_output_test.append([lemmatizer.lemmatize(w,pos = "v") for w in clean_test_words1[i]])
    clean_test_words_str1.append(" ".join(lemmatized_output_test[i]))
    i+=1



#### Lemmatization and stopwords elimination of train set

In [19]:
lemmatizer = WordNetLemmatizer()
lowercase = []
clean_words1 = []
clean_words_str1 = []
lemmatized_output = []
tokenizer = WordPunctTokenizer()

for word in tr['Comment']:
    lowercase.append(tokenizer.tokenize(word.lower()))

stop_words = set(stopwords.words('english'))
    
i=0
for x in range(0,len(lowercase)):
    
    clean_words1.append([token for token in lowercase[x] if not token in stop_words and token.isalpha()])
    lemmatized_output.append([lemmatizer.lemmatize(w,pos = "v") for w in clean_words1[i]])
    clean_words_str1.append(" ".join(lemmatized_output[i]))
    i+=1


#### Perform Naive Bayes classification for word vectors using bigrams and Laplace smoothing

In [20]:
vectorizer = CountVectorizer(ngram_range=(2, 2))
X2 = vectorizer.fit_transform(clean_words_str1)
T2 = vectorizer.transform(clean_test_words_str1)

X_train, Y_train = X2.toarray(), tr['Insult'] 
X_test = T2.toarray()
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])

clf = MultinomialNB(alpha=1.0) # Laplace Smoothing
clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))


print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))


              precision    recall  f1-score   support

           0       0.57      0.93      0.70      1158
           1       0.75      0.25      0.37      1077

    accuracy                           0.60      2235
   macro avg       0.66      0.59      0.54      2235
weighted avg       0.66      0.60      0.54      2235

0.5977628635346757
0.5433499462866819


**_We have many comments and words that are very hard to process in the dataframe and that is why the scores did not improve after the lemmatization and the stopwords elimination._**

**e.g. n o tenho a m nima paci ncia pra esse tipo**

**_Lemmatization can only apply to a certain part of speech and that is why instead of improving the classification i actually makes it more difficult._**

**e.g. "is" becomes "be" but "oranges" doesn't become "orange"**

## TF-IDF

In [21]:
tfidf_vectorizer = TfidfVectorizer() 
tfidf = tfidf_vectorizer.fit_transform(clean_words_str)

tfidf_test = tfidf_vectorizer.transform(clean_test_words_str)

### Part-of-speech train

In [22]:
freq = []

for i in range(0,len(clean_words_str)):
    noun = 0
    verb = 0
    adv = 0
    adj = 0
    s = 0

    text = word_tokenize(clean_words_str[i])
    tagged = pos_tag(text)
    
    for word,j in tagged:
        s += 1
        if(j.startswith('NN')):
            noun += 1
        if(j.startswith('VB')):
            verb += 1
        if(j.startswith('RB')):
            adv += 1
        if(j.startswith('JJ')):
            adj += 1


    if(s==0):
        freq.append([s,s,s,s])
    else:
        freq.append([noun/s, verb/s,adv/s,adj/s])


### Part-of-speech test

In [23]:
freq1 = []

for i in range(0,len(clean_test_words_str)):
    noun = 0
    verb = 0
    adv = 0
    adj = 0
    s = 0

    text = word_tokenize(clean_test_words_str[i])
    tagged = pos_tag(text)
    
    for word,j in tagged:
        s += 1
        if(j.startswith('NN')):
            noun += 1
        if(j.startswith('VB')):
            verb += 1
        if(j.startswith('RB')):
            adv += 1
        if(j.startswith('JJ')):
            adj += 1


    if(s==0):
        freq1.append([s,s,s,s])
    else:
        freq1.append([noun/s, verb/s,adv/s,adj/s])


### Combination of tf-idf and part-of-speech

In [24]:
X = tfidf.toarray().tolist()

for i in range(0,len(clean_words_str)):
    for j in range(0,4): 
        X[i].append(freq[i][j])
        
X_train = np.array(X)

In [25]:
Xt = tfidf_test.toarray().tolist()

for i in range(0,len(clean_test_words_str)):
    for j in range(0,4): 
        Xt[i].append(freq1[i][j])
        
X_test = np.array(Xt)

### SVM

In [26]:
Y_train = tr['Insult']
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])

clf = SVC(kernel='linear')
clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))


print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))


              precision    recall  f1-score   support

           0       0.66      0.89      0.76      1158
           1       0.81      0.50      0.62      1077

    accuracy                           0.70      2235
   macro avg       0.73      0.70      0.69      2235
weighted avg       0.73      0.70      0.69      2235

0.7038031319910515
0.6913780820909229


### Random Forest

In [27]:
Y_train = tr['Insult']
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])

clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))


print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))


              precision    recall  f1-score   support

           0       0.59      0.97      0.73      1158
           1       0.89      0.27      0.41      1077

    accuracy                           0.63      2235
   macro avg       0.74      0.62      0.57      2235
weighted avg       0.73      0.63      0.58      2235

0.6313199105145414
0.5779406303263984


## Beat The Benchmark

### Further Dataframe cleaning to improve f1 score

In [29]:
tr['Comment'] = tr['Comment'].str.replace(r"\\'","'",regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\@","",regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"n\'t", " not", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'re", " are", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'s", " is", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'d", " would", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'ll", " will", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'t", " not", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'ve", " have", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'m", " am", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"\'m", " am", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"#", " ", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"%", " ", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"&", " ", regex = True)
tr['Comment'] = tr['Comment'].str.replace(r"*", " ", regex = True)

In [31]:
lowercase = []
clean_words = []
clean_words_str = []
tokenizer = WordPunctTokenizer()
for word in tr['Comment']:
    lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(lowercase)):
    
    clean_words.append([token for token in lowercase[x] if token.isalpha() and len(token) > 1])
    clean_words_str.append(" ".join(clean_words[i]))
    i+=1

In [32]:
test_lowercase = []
clean_test_words = []
clean_test_words_str = []
tokenizer = WordPunctTokenizer()
for word in te['Comment']:
    test_lowercase.append(tokenizer.tokenize(word.lower()))

i=0
for x in range(0,len(test_lowercase)):
    
    clean_test_words.append([token for token in test_lowercase[x] if token.isalpha() and len(token) > 1])
    clean_test_words_str.append(" ".join(clean_test_words[i]))
    i+=1

In [33]:
vectorizer = CountVectorizer(analyzer='word')
X2 = vectorizer.fit_transform(clean_words_str)
T2 = vectorizer.transform(clean_test_words_str)

X_train, Y_train = X2, tr['Insult'] 
X_test = T2
Y_test = pd.read_csv('data/impermium_verification_labels.csv',usecols=[1])

clf = SVC(kernel='linear')
clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)

confusion_matrix(Y_test,Y_pred) 
print(classification_report(Y_test,Y_pred))


print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.69      0.84      0.75      1158
           1       0.77      0.59      0.67      1077

    accuracy                           0.72      2235
   macro avg       0.73      0.71      0.71      2235
weighted avg       0.73      0.72      0.71      2235

0.7172259507829978
0.7122983972610082


- F1 score improves in the SVM classifier when we use the CountVectorizer instead of the TF-IDF
- The score also improved slightly because we removed some unnecessary characters

## Conclusion

Due to the complexity of the dataframe it is quite difficult to achieve really high scores. Also, in spite of removing stopwords or lemmatizing the data, we didn't see any imrovements and in some cases we noticed a slight downgrade

On the other hand we saw a big upgrade from GaussianNB to Laplace smoothing (the explanation is right below the related code). F1 score went from 0.518 to 0.68. We then saw a smaller upgrade when we used bigrams where the f1 score went from 0.518 to 0.56.